# Collect solar generation data relevant to the NEM
- panels are single axis tracker
- all spatial points are averaged across the REZs

In [1]:
import sys
sys.path.append('/home/548/cd3022/aus-historical-solar-droughts/code/python/scripts')
import utils_V2
import os

from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import xarray as xr

In [2]:
start_date = sys.argv[1]
end_date = sys.argv[2]

# start_date = '1-1-2022'
# end_date = start_date

In [3]:
# REZ mask
mask_file = "/home/548/cd3022/aus-historical-solar-droughts/data/boundary_files/REZ_mask.npz"
loaded_mask = np.load(mask_file)
mask = loaded_mask["mask"]

In [4]:
# datetime object for finding files
dir_dt = datetime.strptime(start_date, "%d-%m-%Y")
end_dt = datetime.strptime(end_date, "%d-%m-%Y")

# num files for the loop, and for making output file shape
num_days = (end_dt - dir_dt).days+1
num_files = 103 * num_days # 103 files in each directory

# same lat/lon for all data
lat_len = 1726
lat_vals = np.linspace(-44.5, -10.0, lat_len).astype(np.float32)
lon_len = 2214
lon_vals = np.linspace(112, 156.26, lon_len).astype(np.float32)
lon_grid, lat_grid = np.meshgrid(lon_vals, lat_vals)
lat_grid = lat_grid[mask]
lon_grid = lon_grid[mask]

times = [] # collect times while iterating and add at the end

# make output array with correct shape first, for efficiency
output_data = xr.DataArray(
    np.full((num_files), np.nan),
    dims=('time'),
    name='capacity_factor'
)

In [5]:
# loop through all files in date range
for i in range(num_files):
    dataset = utils_V2.get_irradiance_dataset(dir_dt=dir_dt)

    if dataset is not None: # only look at data if file exists

        # irradiance data for capacity factor calculations
        REZ_ghi = dataset['surface_global_irradiance'].squeeze(dim="time").values[mask]
        REZ_dni = dataset['direct_normal_irradiance'].squeeze(dim="time").values[mask]
        REZ_dhi = dataset['surface_diffuse_irradiance'].squeeze(dim="time").values[mask]
        dataset.close()
        
        # time, lat, and lon for solar angles needed for capacity factor calculations
        utc_dt = dir_dt - timedelta(hours=5, minutes=30)
        # time = pd.Timestamp(f'{utc_dt.year}-{utc_dt.month}-{utc_dt.day} {utc_dt.hour}:{utc_dt.minute}:00', tz='UTC') # for solar calcs
        time = pd.Timestamp(utc_dt, tz='UTC')
        
        # calculate capacity factors using pvlib
        capacity_factors = utils_V2.solar_cf(
            pv_model = 'Canadian_Solar_CS5P_220M___2009_',
            inverter_model = 'ABB__MICRO_0_25_I_OUTD_US_208__208V_',
            ghi=REZ_ghi,
            dni=REZ_dni,
            dhi=REZ_dhi,
            time=time,
            lat_grid=lat_grid,
            lon_grid=lon_grid
        )  
        
        if np.all(np.isnan(capacity_factors)):  
            output_data[i] = np.nan  # or any default value you prefer
        else:
            output_data[i] = np.nanmean(capacity_factors)
    
    times.append(np.datetime64(utc_dt))  # for output file

    if dir_dt.strftime('%H%M') == '1700': # last file of day
        dir_dt = dir_dt + timedelta(hours=7)
    else:
        dir_dt = dir_dt + timedelta(minutes=10)
    
output_data = output_data.assign_coords(time=times) # add time values

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/pvlib/spa.py:753: RuntimeWarning: invalid value encountered in arcsin
  e0 = np.degrees(np.arcsin(


File not found: /g/data/rv74/satellite-products/arc/der/himawari-ahi/solar/p1s/v1.1/2022/01/01/IDE00326.202201010240.nc.


/jobfs/136901725.gadi-pbs/ipykernel_44138/2399254984.py:38: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  output_data = output_data.assign_coords(time=times) # add time values


In [7]:
output_file = f'/g/data/er8/users/cd3022/solar_drought/NEM_hourly_capacity_factors/{start_date}___{end_date}.nc'
os.makedirs(os.path.dirname(output_file), exist_ok=True)
output_data.to_netcdf(output_file)

In [10]:
if __name__ == '__main__':
    
    NOTEBOOK_PATH="/home/548/cd3022/aus-historical-solar-droughts/code/python/notebooks/NEM-data-collect.ipynb"
    SCRIPT_PATH="/home/548/cd3022/aus-historical-solar-droughts/code/python/scripts/NEM-data-collect"
    
    !jupyter nbconvert --to script {NOTEBOOK_PATH} --output {SCRIPT_PATH}

[NbConvertApp] Converting notebook /home/548/cd3022/aus-historical-solar-droughts/code/python/notebooks/NEM-data-collect.ipynb to script
[NbConvertApp] Writing 3875 bytes to /home/548/cd3022/aus-historical-solar-droughts/code/python/scripts/NEM-data-collect.py
